# Weather data fetch tests
This notebook will walk through how to get forecasted weather data for a location. 

the weather api used is from weather.gov: https://www.weather.gov/documentation/services-web-api

# the weather fetching process is:
1.  get the lat long for the sites
2.  get the grid points
3.  call the for the appropriate forecast 


# Get lat long for each site
Using google maps, I typed in the locations and then asked for the lat and long of that spot on the map. Here are the lat long coordinates for our locations
![caltech](caltech_lat_long.png)

In [1]:
import requests
import pandas as pd
import json

caltech_lat =   34.134785646454844
caltech_lon = -118.11691382579643

jpl_lat = 34.20142342818471
jpl_lon = -118.17126565774107

office_lat = 37.33680466796926
office_lon = -121.90743423142634

# get grid points
The way the weather api system works is by finding the station's grid points that are closest to the lat long coordinates. The code below gets the grid x and grid y from the lat long coordinates

In [2]:
def get_grid_points(latitude, longitude):
    url = f'https://api.weather.gov/points/{latitude},{longitude}'
    print(url)
    
    r = requests.get(url)
    
    print('status code:', r.status_code)
    payload = r.json()
    
    gridId = payload['properties']['gridId']
    gridX = payload['properties']['gridX']
    gridY = r.json()['properties']['gridY']
    return gridId, gridX, gridY
cal = get_grid_points(caltech_lat, caltech_lon)
off = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


In [3]:
ct_grid, ct_grid_x, ct_grid_y = get_grid_points(caltech_lat, caltech_lon)
jpl_grid, jpl_grid_x, jpl_grid_y = get_grid_points(jpl_lat, jpl_lon)
office_grid, office_grid_x, office_grid_y = get_grid_points(office_lat, office_lon)

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/points/34.20142342818471,-118.17126565774107
status code: 200
https://api.weather.gov/points/37.33680466796926,-121.90743423142634
status code: 200


# get forecast
Once we have the office, gridx and gridy for the location, we can get three different kinds of weather forecasts

    forecast - forecast for 12h periods over the next seven days
    forecastHourly - forecast for hourly periods over the next seven days
    forecastGridData - raw forecast data over the next seven days



In [4]:
'''
def get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):
    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:
        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )
        pass
    
    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',
                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',
                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}
    
    url = forecast_urls[forecast_type]
    print(url)
    r = requests.get(url)
    return r.json()
'''


"\ndef get_weather_forecast(office, gridX, gridY, forecast_type='forecast'):\n    if forecast_type not in ['12_hour', '7day_hourly', '7day_grid']:\n        print(f'{forecast_type} is not a valid option. please select 12_hour, 7day_hourly, or 7day_grid', )\n        pass\n    \n    forecast_urls = {'12_hour':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast',\n                     '7day_hourly':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly',\n                     '7day_grid':f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'}\n    \n    url = forecast_urls[forecast_type]\n    print(url)\n    r = requests.get(url)\n    return r.json()\n"

In [5]:
def get_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast'
    print(url)
    r = requests.get(url, headers={'Accept': 'application/geo+json'})
    return r.json()

In [6]:
def get_hourly_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/hourly'
    print(url)
    r = requests.get(url)
    return r.json()

In [7]:
def get_raw_weather_forecast(office, gridX, gridY):
    url = f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}'
    print(url)
    r = requests.get(url)
    return r.json()

In [8]:
#forecast_12_hour = get_weather_forecast('LOX', 160, 48, '12_hour')
#forecast_7day_hourly = get_weather_forecast('LOX', 160, 48, '7day_hourly')
#forecast_7day_raw = get_weather_forecast('LOX', 160, 48, '7day_grid')

In [9]:
#forecast_7day_hourly

In [10]:
ct_forecast = get_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_forecast

https://api.weather.gov/gridpoints/LOX/160,48/forecast


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.1416245, 34.1484987],
    [-118.13699860000001, 34.1262836],
    [-118.11015800000001, 34.1301103],
    [-118.11477870000002, 34.1523259],
    [-118.1416245, 34.1484987]]]},
 'properties': {'updated': '2024-04-06T16:54:21+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-04-06T19:17:48+00:00',
  'updateTime': '2024-04-06T16:54:21+00:00',
  'validTimes': '2024-04-06T10:00:00+00:00/P7DT15H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 227.076},
  'periods': [{'number': 1,
    'name': 'This Afternoon',
    'startTime': '2024-04-06T12:00:00-07:00',
    'endTime': '2024-04-0

In [11]:
def create_forecast_df(json_forecast):
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    forecast_df['startTime'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df['endTime'] = pd.to_datetime(forecast_df['endTime'], utc=True)
    return forecast_df

In [12]:
ct_forecast_df = create_forecast_df(ct_forecast)
ct_forecast_df

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,This Afternoon,2024-04-06 19:00:00+00:00,2024-04-07 01:00:00+00:00,True,63,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 47}",5 to 10 mph,SW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 63. Southwest wind 5 t..."
1,2,Tonight,2024-04-07 01:00:00+00:00,2024-04-07 13:00:00+00:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 81}",5 to 10 mph,N,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 44. North wind..."
2,3,Sunday,2024-04-07 13:00:00+00:00,2024-04-08 01:00:00+00:00,True,65,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.777777...","{'unitCode': 'wmoUnit:percent', 'value': 82}",5 to 10 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 65. South southwest wi..."
3,4,Sunday Night,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,45,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 79}",5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 45. Northwest ..."
4,5,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,72,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 4.444444...","{'unitCode': 'wmoUnit:percent', 'value': 81}",5 to 15 mph,N,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 72. North wind 5 to 15..."
5,6,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,49,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 1.666666...","{'unitCode': 'wmoUnit:percent', 'value': 38}",5 to 15 mph,N,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 49."
6,7,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,78,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 60}",5 to 10 mph,NNW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 78."
7,8,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,54,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 53}",5 to 10 mph,N,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 54."
8,9,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,82,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 52}",5 to 10 mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 82."
9,10,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 59}",5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 57."


In [13]:
ct_hourly_forecast = get_hourly_weather_forecast(ct_grid, ct_grid_x, ct_grid_y)
ct_hourly_forecast

https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.1416245, 34.1484987],
    [-118.13699860000001, 34.1262836],
    [-118.11015800000001, 34.1301103],
    [-118.11477870000002, 34.1523259],
    [-118.1416245, 34.1484987]]]},
 'properties': {'updated': '2024-04-06T16:54:21+00:00',
  'units': 'us',
  'forecastGenerator': 'HourlyForecastGenerator',
  'generatedAt': '2024-04-06T19:15:05+00:00',
  'updateTime': '2024-04-06T16:54:21+00:00',
  'validTimes': '2024-04-06T10:00:00+00:00/P7DT15H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 227.076},
  'periods': [{'number': 1,
    'name': '',
    'startTime': '2024-04-06T12:00:00-07:00',
    'endTime': '2024-04-06T13:00:00-07:00

In [14]:
import numpy as np
def create_hourly_forecast_df(json_forecast):
    # grab each hourly entry in the forecast
    forecast = json_forecast['properties']['periods']
    forecast_df = pd.DataFrame(forecast)
    
    # convert startTime to UTC datetime and set as index
    forecast_df['time'] = pd.to_datetime(forecast_df['startTime'], utc=True)
    forecast_df = forecast_df.set_index('time')
    
    # convert temperature to degF and degC
    forecast_df.rename(columns = {'temperature':'temperature_degF'}, inplace = True) 
    forecast_df['temperature_degC'] = forecast_df['temperature_degF'].apply(lambda x: np.round((x - 32) * 5.0/9.0, 2))
    
    # convert dewpoint temperature to degF and degC
    forecast_df['dewpoint_degC'] = forecast_df['dewpoint'].apply(lambda x: np.round(x['value'], 2))
    forecast_df['dewpoint_degF'] = forecast_df['dewpoint_degC'].apply(lambda x: np.round(x * 9.0/5.0 + 32, 2))
    
    # extract value from windSpeed column and convert to int
    forecast_df['windSpeed_mph'] = forecast_df['windSpeed'].apply(lambda x: int(x.split(" mph")[0]))
    
    # extract value from json formatted columns
    forecast_df['probabilityOfPrecipitationPercent'] = forecast_df['probabilityOfPrecipitation'].apply(lambda x: x['value'])
    forecast_df['relativeHumidityPercent'] = forecast_df['relativeHumidity'].apply(lambda x: x['value'])
    
    # drop columns that are no longer needed
    return forecast_df.drop(columns=['startTime', 'endTime', 'windSpeed', 'number', 'name', 'detailedForecast', 'dewpoint', 'probabilityOfPrecipitation', 'relativeHumidity', 'temperatureTrend', 'temperatureUnit'])

In [15]:
ct_hourly_forecast_df = create_hourly_forecast_df(ct_hourly_forecast)
ct_hourly_forecast_df

,isDaytime,temperature_degF,windDirection,icon,shortForecast,temperature_degC,dewpoint_degC,dewpoint_degF,windSpeed_mph,probabilityOfPrecipitationPercent,relativeHumidityPercent
time,,,,,,,,,,,
2024-04-06 19:00:00+00:00,True,56,S,"https://api.weather.gov/icons/land/day/skc,0?s...",Sunny,13.33,1.67,35.01,5,0,45
2024-04-06 20:00:00+00:00,True,58,S,"https://api.weather.gov/icons/land/day/skc,0?s...",Sunny,14.44,1.11,34.00,5,0,40
2024-04-06 21:00:00+00:00,True,62,SW,"https://api.weather.gov/icons/land/day/skc,0?s...",Sunny,16.67,2.78,37.00,10,0,40
2024-04-06 22:00:00+00:00,True,63,SW,"https://api.weather.gov/icons/land/day/skc,0?s...",Sunny,17.22,3.89,39.00,10,0,42
2024-04-06 23:00:00+00:00,True,63,SW,"https://api.weather.gov/icons/land/day/skc,0?s...",Sunny,17.22,5.00,41.00,10,0,44
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-13 02:00:00+00:00,False,72,SW,"https://api.weather.gov/icons/land/night/sct,1...",Partly Cloudy,22.22,9.44,48.99,10,1,46
2024-04-13 03:00:00+00:00,False,68,SSW,"https://api.weather.gov/icons/land/night/sct,1...",Partly Cloudy,20.00,10.00,50.00,10,1,53
2024-04-13 04:00:00+00:00,False,65,SSW,"https://api.weather.gov/icons/land/night/sct,1...",Partly Cloudy,18.33,10.00,50.00,10,1,60


In [16]:
jpl_forecast = get_weather_forecast(jpl_grid, jpl_grid_x, jpl_grid_y)
jpl_forecast_df = create_forecast_df(jpl_forecast)
jpl_forecast_df

https://api.weather.gov/gridpoints/LOX/159,51/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,Today,2024-04-06 18:00:00+00:00,2024-04-07 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 54}",10 mph,SW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 61. Southwest wind aro..."
1,2,Tonight,2024-04-07 01:00:00+00:00,2024-04-07 13:00:00+00:00,False,43,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 72}",5 to 10 mph,NNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 43. North nort..."
2,3,Sunday,2024-04-07 13:00:00+00:00,2024-04-08 01:00:00+00:00,True,63,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 80}",5 to 10 mph,WSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 63. West southwest win..."
3,4,Sunday Night,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,44,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 79}",5 to 10 mph,NNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 44. North nort..."
4,5,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,69,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.777777...","{'unitCode': 'wmoUnit:percent', 'value': 76}",5 to 20 mph,N,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 69. North wind 5 to 20..."
5,6,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,49,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 1.666666...","{'unitCode': 'wmoUnit:percent', 'value': 38}",10 to 20 mph,N,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 49."
6,7,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,75,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 3.888888...","{'unitCode': 'wmoUnit:percent', 'value': 57}",10 mph,NW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 75."
7,8,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,54,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 2.777777...","{'unitCode': 'wmoUnit:percent', 'value': 48}",5 to 10 mph,N,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 54."
8,9,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,79,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5}","{'unitCode': 'wmoUnit:percent', 'value': 47}",10 mph,SSW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 79."
9,10,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,57,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 54}",5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 57."


In [17]:
office_forecast = get_weather_forecast(office_grid, office_grid_x, office_grid_y)
office_forecast_df = create_forecast_df(office_forecast)
office_forecast_df

https://api.weather.gov/gridpoints/MTR/99,82/forecast


,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast
0,1,This Afternoon,2024-04-06 19:00:00+00:00,2024-04-07 01:00:00+00:00,True,61,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 4.444444...","{'unitCode': 'wmoUnit:percent', 'value': 52}",6 to 9 mph,NW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 61. Northwest w..."
1,2,Tonight,2024-04-07 01:00:00+00:00,2024-04-07 13:00:00+00:00,False,41,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 5.555555...","{'unitCode': 'wmoUnit:percent', 'value': 92}",3 to 10 mph,W,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 41. West wind..."
2,3,Sunday,2024-04-07 13:00:00+00:00,2024-04-08 01:00:00+00:00,True,63,F,falling,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.222222...","{'unitCode': 'wmoUnit:percent', 'value': 91}",2 to 13 mph,W,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny. High near 63, with temperatures ..."
3,4,Sunday Night,2024-04-08 01:00:00+00:00,2024-04-08 13:00:00+00:00,False,41,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.666666...","{'unitCode': 'wmoUnit:percent', 'value': 97}",5 to 13 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 41. Northwest ..."
4,5,Monday,2024-04-08 13:00:00+00:00,2024-04-09 01:00:00+00:00,True,71,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 6.111111...","{'unitCode': 'wmoUnit:percent', 'value': 94}",5 to 15 mph,NW,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 71. Northwest wind 5 t..."
5,6,Monday Night,2024-04-09 01:00:00+00:00,2024-04-09 13:00:00+00:00,False,45,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 7.777777...","{'unitCode': 'wmoUnit:percent', 'value': 83}",3 to 15 mph,NNE,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 45."
6,7,Tuesday,2024-04-09 13:00:00+00:00,2024-04-10 01:00:00+00:00,True,77,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 8.888888...","{'unitCode': 'wmoUnit:percent', 'value': 83}",3 to 10 mph,ENE,https://api.weather.gov/icons/land/day/skc?siz...,Sunny,"Sunny, with a high near 77."
7,8,Tuesday Night,2024-04-10 01:00:00+00:00,2024-04-10 13:00:00+00:00,False,50,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 10}","{'unitCode': 'wmoUnit:percent', 'value': 80}",2 to 10 mph,NNE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 50."
8,9,Wednesday,2024-04-10 13:00:00+00:00,2024-04-11 01:00:00+00:00,True,82,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 11.66666...","{'unitCode': 'wmoUnit:percent', 'value': 78}",3 to 8 mph,N,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 82."
9,10,Wednesday Night,2024-04-11 01:00:00+00:00,2024-04-11 13:00:00+00:00,False,53,F,None,"{'unitCode': 'wmoUnit:percent', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': 13.88888...","{'unitCode': 'wmoUnit:percent', 'value': 84}",3 to 8 mph,NNE,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 53."
